# Import neeed files and Library’s 

In [1]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import requests


        
        

# Get top 25 cities by population from web site https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population

Extract Rank, City, state, Population, latitude and longitude then add to Data Frame 


In [2]:


page = requests.get("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")

Rank = 0 

soup = BeautifulSoup(page.content, 'lxml')


# Create data Frame 
#Population (2018 estimate)

df = pd.DataFrame(columns =["Rank", "City", "State", "Population", "latitude", "longitude"]) 


for items in soup.find("table",{"class":"wikitable sortable"}).find_all("tr")[1:-1]:
    data = [' '.join(item.text.split()) for item in items.find_all(['th','td'])]
    
    if (Rank < 25): 
        #print (data)
        Rank = int((data[0]))
        City = (data[1])
        State = (data[2])
        Population = (data[3])
        Location = str((data[10]))
        
        # format City name to remove possible "[]" 
        
        if "[" in City: 
            p = City.index("[")
            #print (p)
            City = City[ : (p)]
            #print("found one")
        
        

        # format loaction to get Latitude and Logitude and convert to number 
        
        s = Location.index("/")
        f = Location.index("(")
        f1 = Location[(s + 1) : (f -1 )]
        s1 = f1.index("/")
        final_location = (f1 [(s1 + 2) : f])
        L1, L2 = (final_location.replace(";","")).split()
        latitude = float(L1)
        longitude = float(L2.replace("\ufeff", ""))
        #print (latitude, longitude)
        
        # add to Data Frame 
        
        df = df.append({"Rank" : Rank ,"City" : City, "State" : State, "Population": Population, "latitude" : latitude, "longitude" : longitude}, ignore_index=True)

print(df.head(5))
#print (df.shape)
    
        
        
        

  Rank         City       State Population  latitude  longitude
0    1     New York    New York  8,398,748   40.6635   -73.9387
1    2  Los Angeles  California  3,990,456   34.0194  -118.4108
2    3      Chicago    Illinois  2,705,994   41.8376   -87.6818
3    4      Houston       Texas  2,325,502   29.7866   -95.3909
4    5      Phoenix     Arizona  1,660,272   33.5722  -112.0901


Install Library’s needed for maps 

In [14]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
from folium.features import DivIcon
#from folium import plugins
print("Imported")



Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.1               |             py_0         575 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

# Map all 25 cities

In [39]:
map = folium.Map(location=[39, -94], zoom_start=5)

Add markers for top 25

In [40]:
for lat, lng, City, State, Rank in \
 zip(df['latitude'], df['longitude'], df['City'], df['State'], df['Rank']):
 #L = (df[City] + " " + df[Rank])
 label = '{}, {}'.format(City, State)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker(
 [lat, lng],
 radius=3,
 popup=label,
 color='blue',
 fill=True,
 fill_color='#3186cc',
 fill_opacity=0.2,
 parse_html=False).add_to(map)

# folium.map.Marker(
   #  [lat, lng],
    # icon =DivIcon(
     #icon_size=(150,36),
     #icon_anchor=(0,0),
    # html='<div style="font-size: 12pt">%s</div>' % Rank,
    # )
 # ).add_to(map)
    
    
    

map


# Get Burger information for each city 

Foursquare Credentials and Version

In [84]:
CLIENT_ID = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 35
#top_picks = 3
 
# which ciy from to map as demo city 
city_map = 2 


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
CLIENT_SECRET:XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


In [66]:
search_query = 'Burger'

print("searching for ", search_query )

searching for  Burger


# Go through each city and get burger restaurants in each city up to 35, then sort by number of likes 
# Select the top two for each city  


In [67]:
from pandas.io.json import json_normalize

Df_Final = pd.DataFrame(columns =["Name", "City", "State", "Address", "Postal_Code", "ID",  "Likes"])
Total = 0

df_map = pd.DataFrame(columns =["Name", "Latitude", "Longitude"])

for city in range(25):
    test = (city )
    
   
    Df_City = pd.DataFrame(columns =["Name", "City", "State", "Address", "Postal_Code", "ID",  "Likes" ,"Latitude", "Longitude"])
   
    latitude = df.loc[city][4]
    longitude = df.loc[city][5]

    #print(latitude, longitude)
    
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query,  LIMIT)
  
    results = requests.get(url).json()
    
   # print(results)
    
    venues = results['response']['venues']
  
    # Change into dataframe
    dataframe = json_normalize(venues)

    #print(dataframe.head)

    for item in range (len(dataframe)):
    
        N_loc = dataframe.columns.get_loc("name")
        C_loc = dataframe.columns.get_loc("location.city")
        PC_loc = dataframe.columns.get_loc("location.postalCode")
        Add_loc = dataframe.columns.get_loc("location.address")
        ST_loc = dataframe.columns.get_loc("location.state")
        ID_loc = dataframe.columns.get_loc("id")
        
        lat_loc = dataframe.columns.get_loc("location.lat")
        lng_loc = dataframe.columns.get_loc("location.lng")

        name = dataframe.loc[item][N_loc]
        city = dataframe.loc[item][C_loc]
        state = dataframe.loc[item][ST_loc]
        address = dataframe.loc[item][Add_loc]
        postal_code = dataframe.loc[item][PC_loc]
        ID = dataframe.loc[item][ID_loc]
        lat = dataframe.loc[item][lat_loc]
        log = dataframe.loc[item][lng_loc]
        
        
        
# ****************************************   Get number of likes ******************************************************************


        likes_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(ID, CLIENT_ID, CLIENT_SECRET, VERSION)

        like_results = requests.get(likes_url).json()
            #print(like_results)

        Likes = like_results['response']['likes']

            # Change into dataframe
        Like_dataframe = json_normalize(Likes)
            #Like_dataframe.head()
        N_Likes = Like_dataframe.loc[0][0]


            #print(N_Likes)

        #print("City DF", " ", name, " ",city, " ", state, ' ', address, " ", postal_code, " ", ID, " ", N_Likes)

        Df_City = Df_City.append({"Name" : name ,"City" : city, "State" : state, "Address": address, "Postal_Code" : postal_code, "ID" :ID, "Likes" : N_Likes, "Latitude" : lat, "Longitude": log}, ignore_index=True)

    Df_City = Df_City.sort_values("Likes", ascending=False).reset_index(drop=True)
 

    new_name = ""
        
    number_for_city = 0 
        
    for res in range (len(Df_City )):

        #Get top two Unique 

        name2 = Df_City.loc[res][0]
        city2 = Df_City.loc[res][1]
        state2 = Df_City.loc[res][2]
        address2 = Df_City.loc[res][3]
        postal_code2 = Df_City.loc[res][4]
        ID2 = Df_City.loc[res][5]
        Likes2 = Df_City.loc[res][6]
        lat = Df_City.loc[res][7]
        log = Df_City.loc[res][8]
            
        

        #print (name2 , city2 , state2 ,  address2, postal_code2, ID2,  Likes2)

        #print(new_name, "  ", name2)
        if new_name != name2 and number_for_city != 2:

            new_name = name2
            number_for_city = number_for_city  + 1 
            Total = Total + 1 

            #print (name2 , city2 , state2 ,  address2, postal_code2, ID2,  Likes2)

            Df_Final = Df_Final.append({"Name" : name2 ,"City" : city2, "State" : state2, "Address": address2, "Postal_Code" : postal_code2, "ID" : ID2,  "Likes" : Likes2}, ignore_index=True)
            
            #print("added 1 for City :", city2, " Total number is now :", len(Df_Final), " ", "  Number for city :", number_for_city)


            
            
# add maping information for mapping City if needed ***************************************************

#Get information Name, latitude, longitude for a city to map as example save in Data frame

            #print (test, " ", city_map)
            if city_map == test:

               # print ("adding :", name, " " ,lat, " ", log)
                df_map = df_map.append({"Name" : name2, "Latitude" : lat, "Longitude": log}, ignore_index=True)

# End add mapping information ***************************************************************************
                                    
                                    
print("End of Run")

#dataframe
Df_Final  
#Df_City      
#TP_DF    
#df_map 

End of Run


,Name,City,State,Address,Postal_Code,ID,Likes
0,Burger Joint,New York,NY,119 W 56th St,10019,41b3a100f964a5206c1e1fe3,2290
1,5 Napkin Burger,New York,NY,630 9th Ave,10036,49d8c05df964a520c95d1fe3,1062
2,In-N-Out Burger,Los Angeles,CA,7009 W Sunset Blvd,90028,49e3c0c1f964a520c7621fe3,1897
3,Umami Burger,Santa Monica,CA,525 Broadway,90401,4bad0a75f964a52081273be3,559
4,DMK Burger Bar,Chicago,IL,2954 N Sheffield Ave,60657,4af9b20df964a5206e1322e3,561
5,Burger Bar,Chicago,IL,1578 N Clybourn Ave,60642,4c002683c30a2d7f3b1a121d,335
6,Bubba's Texas Burger Shack,Houston,TX,5230 Westpark Dr,77056,4b05c07cf964a52071e222e3,150
7,The Burger Joint,Houston,TX,2703 Montrose Blvd,77006,563932de498eb0f7a0c4ccdb,149
8,In-N-Out Burger,Tempe,AZ,920 E Playa del Norte Dr,85281,4a3d9ccdf964a5208aa21fe3,574
9,The Habit Burger Grill,Phoenix,AZ,745 E Glendale Ave,85020,4e6bc4edae609509560e72a2,66


# Top Pick - Find two top museums in each city 

In [68]:

TP_Final = pd.DataFrame(columns =["Name", "City", "State", "Address", "Postal_Code", "ID"])

L = 2
cat = "4bf58dd8d48988d181941735"
park = "4bf58dd8d48988d163941735"


for city in range(25):
    
    test = (city)
    
    Df_City = pd.DataFrame(columns =["Name", "City", "State", "Address", "Postal_Code", "ID",  "Likes"])
   
    latitude = df.loc[city][4]
    longitude = df.loc[city][5]

    #print(latitude, longitude)        
            
            
# ***************************   Top ************************************

    if city == 14:  # look at parks in San Francisco no museums found 
        look_at = park
    else:
        look_at = cat

    url_top = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, look_at, L)

    top_results = requests.get(url_top).json()

    #print(top_results)

    top_venues = top_results['response']['groups'][0]['items']
        #top_venues = results['response']['venues']

        # Change into dataframe
    TP_DF = json_normalize(top_venues)

    #print(TP_DF)

    for item2 in range (len(TP_DF)):
        #print(len(TP_DF.columns), " ***")



        N_loc1 = TP_DF.columns.get_loc("venue.name")    
        C_loc1 = TP_DF.columns.get_loc("venue.location.city")
        PC_loc1 = TP_DF.columns.get_loc("venue.location.postalCode")
        Add_loc1 = TP_DF.columns.get_loc("venue.location.address")
        ST_loc1 = TP_DF.columns.get_loc("venue.location.state")
        ID_loc1 = TP_DF.columns.get_loc("venue.id")
        
        

        #print (list(dataframe.columns.values) )
        name3 = TP_DF.loc[item2][N_loc1]
        city3 = TP_DF.loc[item2][C_loc1]
        state3 = TP_DF.loc[item2][ST_loc1 ]
        address3 = TP_DF.loc[item2][Add_loc1]
        postal_code3 = TP_DF.loc[item2][PC_loc1]
        ID3 = TP_DF.loc[item2][ID_loc1]



        TP_Final = TP_Final.append({"Name" : name3 ,"City" : city3, "State" : state3, "Address": address3, "Postal_Code" : postal_code3, "ID" :ID3}, ignore_index=True)

        # Get information Name, latitude, longitude for a city to map as example save in Data frame
        #print (city_map, " " , test)
        if city_map == test:


            N_name_loc = TP_DF.columns.get_loc("venue.name")
            lat_loc = TP_DF.columns.get_loc("venue.location.lat")
            lng_loc = TP_DF.columns.get_loc("venue.location.lng")



            Nname = TP_DF.loc[item2][N_name_loc]
            lat = TP_DF.loc[item2][lat_loc]
            log = TP_DF.loc[item2][lng_loc]


            df_map = df_map.append({"Name" : Nname, "Latitude" : lat, "Longitude": log}, ignore_index=True)

#TP_DF
TP_Final


,Name,City,State,Address,Postal_Code,ID
0,Jewish Children's Museum,Brooklyn,NY,792 Eastern Pkwy,11213,4a42e2c9f964a52059a61fe3
1,Brooklyn Children's Museum,New York,NY,145 Brooklyn Ave,NaN,49cfbb2df964a520c05a1fe3
2,Museum of Jurassic Technology,Culver City,CA,9341 Venice Blvd,90232,4287e280f964a5203e231fe3
3,Center for Land Use Interpretation,Culver City,CA,9331 Venice Blvd,90232,4b2bb215f964a52048b924e3
4,National Museum of Mexican Art,Chicago,IL,1852 W 19th St,60608,4b0313ebf964a520824c22e3
5,Jane Addams Hull-House Museum,Chicago,IL,800 S Halsted St,60607,4b9aaa7bf964a52034cb35e3
6,Art Car Museum,Houston,TX,140 Heights Blvd,77007,4ac8f41af964a5208dbd20e3
7,Heights Fire Station,Houston,TX,NaN,77008,4bad5415f964a520b4463be3
8,Sahuaro Ranch Park,Glendale,AZ,9802 N 59th Ave,85302,4b51e1a1f964a5206e5927e3
9,Heard Museum,Phoenix,AZ,2301 N Central Ave,85004,4a6b58bbf964a52087ce1fe3


# Print Data frame showing 4 items in sample city to map 

In [69]:
df_map


,Name,Latitude,Longitude
0,DMK Burger Bar,41.936198,-87.654191
1,Burger Bar,41.910375,-87.648493
2,National Museum of Mexican Art,41.856038,-87.672840
3,Jane Addams Hull-House Museum,41.871673,-87.647238


# Map sample city and label three places of interest  

In [83]:
latitude1 = df.loc[city_map][4]
longitude1 = df.loc[city_map][5]

print(latitude1, "  ",longitude1 )



map_city = folium.Map(location=[latitude1, longitude1], zoom_start=15
                     )


for lat, lng, name in \
 zip(df_map['Latitude'], df_map['Longitude'], df_map['Name']):

 label = '{}'.format(name)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker(
 [lat, lng],
 radius=15,
 popup=label,
 color='blue',
 fill=True,
 fill_color='#3186cc',
 fill_opacity=0.2,
 parse_html=False).add_to(map_city)

 folium.map.Marker(
     [lat , lng + .05],
     icon =DivIcon(
     icon_size=(150,36),
     icon_anchor=(0,0),
     html='<div style="font-size: 11pt, font-weight:bold;">%s</div>' % name,
     )
  ).add_to(map_city)
    
    
    

map_city


41.8376    -87.6818
